In [1]:
import pandas as pd
pca_data = pd.read_csv('feature_selected_data.csv',sep = ',')

In [3]:
pca_data.shape

(717129, 31)

In [2]:

from pyspark.ml import feature
from pyspark.ml import clustering
from pyspark.ml import Pipeline
from pyspark.sql import functions as fn
import numpy as np
from pyspark.sql import SparkSession
from pyspark.ml import feature, regression, evaluation, Pipeline
from pyspark.sql import functions as fn, Row
import matplotlib.pyplot as plt
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [3]:
pca_data.fillna(0,inplace = True)
pca_data = pca_data.astype(int)


In [4]:
pca_data.columns = pca_data.columns.str.replace('.','_')

In [5]:

pca_data.rename(columns={'Referral': 'Channel_group_referral', 'referral': 'traffic_source_referral'}, inplace=True)


In [6]:
pca_data.loc[pca_data["Total revenue"]==0,"Total revenue"] = 0
pca_data.loc[pca_data["Total revenue"]!=0,"Total revenue"] = 1

In [7]:
pca_data['Total revenue'].sum()

17658

In [9]:
pca_data.columns

Index(['totals_hits', 'totals_pageviews', 'visitNumber', 'totals_transactions',
       'Channel_group_referral', 'Chrome OS', 'cpm', 'Macintosh', 'Firefox',
       'totals_bounces', 'Linux', 'Direct', 'Windows', 'Organic Search',
       'organic', 'Paid Search', 'True', 'totals_newVisits', 'mobile',
       'Safari', 'cpc', 'iOS', 'Android', 'Display', 'Internet Explorer',
       'desktop', 'traffic_source_referral', 'Edge', 'Chrome', 'Social',
       'Total revenue'],
      dtype='object')

In [10]:
pca_data2['Total revenue'].sum()

4930

In [11]:
pca_df = spark.createDataFrame(pca_data)

In [40]:
len(pca_df.toPandas()['Total revenue'])

717129

In [ ]:
# training_df, validation_df, testing_df = pca_df.randomSplit([0.6, 0.3, 0.1], seed=0)

In [9]:
pipeline_pca = Pipeline(stages=[
    feature.VectorAssembler(inputCols=pca_df.columns[:-1],
                           outputCol='features'),
    feature.StandardScaler(withMean=True,
                           inputCol='features', outputCol='zfeatures'
                          ),
    feature.PCA(k=2, inputCol='zfeatures', outputCol='loadings')
]).fit(pca_df)

In [10]:
principal_components = pipeline_pca.stages[-1].pc.toArray()

In [11]:
pca_list = list(zip(pca_df.columns[:-1], 
         principal_components[:, 0], principal_components[:, 1]))

In [12]:
pca_list

[('totals_hits', -0.11775911608495077, 0.241600085060104),
 ('totals_pageviews', -0.12218528598132353, 0.24740273567295562),
 ('visitNumber', -0.042505008039364495, -0.013762670379592095),
 ('totals_transactions', -0.07732159525735988, 0.12086847927764896),
 ('Channel_group_referral', -0.21310554217862832, -0.0994666321871792),
 ('Chrome OS', -0.07193142580097961, 0.018404539594888816),
 ('cpm', -0.0012951211277594744, -0.09086215711405327),
 ('Macintosh', -0.22055399070702736, 0.038278469312461386),
 ('Firefox', -0.02193499620995402, 0.020493070460500428),
 ('totals_bounces', 0.13676378026634814, -0.21135710377906602),
 ('Linux', -0.0857145736851394, -0.07148714382324226),
 ('Direct', 0.016470184649338408, -0.1431945006279819),
 ('Windows', -0.12831641196936827, 0.021124581491040937),
 ('Organic Search', 0.0930191126712807, 0.4688863738866928),
 ('organic', 0.08381365453391756, 0.4375613316644085),
 ('Paid Search', 0.05907383070676773, -0.1425863203040946),
 ('True', 0.410843055468355

In [15]:
feature_list = []
component1 = []
component2 = []
for i in pca_list:
    feature_list.append(i[0])
    component1.append(i[1])
    component2.append(i[2])

weight1 = 0.5
weight2 = 0.5
weight = [x + y for x, y in zip([abs(i) * weight1 for i in component1],\
                                [abs(i) * weight2 for i in component2])]


In [16]:
pca_list2 = pd.DataFrame(
    {'feature': feature_list,
     'component1': component1,
     'component2': component2
    })

In [17]:
pca_list_df = pca_list2[['component1','component2']]

In [18]:
pca_list_df.head()

,component1,component2
0,-0.117759,0.241600
1,-0.122185,0.247403
2,-0.042505,-0.013763
3,-0.077322,0.120868
4,-0.213106,-0.099467


In [19]:
pca_list_m = pca_list_df.as_matrix()


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [20]:
training_df, validation_df, testing_df = pca_df.randomSplit([0.6, 0.3, 0.1], seed=0)

In [21]:
training_m = training_df.select(training_df.columns[:-1]).toPandas().as_matrix()
validation_m = validation_df.select(validation_df.columns[:-1]).toPandas().as_matrix()
testing_m = testing_df.select(testing_df.columns[:-1]).toPandas().as_matrix()


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [22]:
trainging_2features_m = training_m.dot(pca_list_m)
validation_2features_m = validation_m.dot(pca_list_m)
testing_2features_m = testing_m.dot(pca_list_m)

In [23]:
trainging_2features_df = pd.DataFrame(trainging_2features_m)
validation_2features_df = pd.DataFrame(validation_2features_m)
testing_2features_df = pd.DataFrame(testing_2features_m)

In [24]:
trainging_2features_df.columns = ['component1','component2']
validation_2features_df.columns = ['component1','component2']
testing_2features_df.columns = ['component1','component2']

In [25]:
trainging_2features_df['Total revenue'] = training_df.toPandas()['Total revenue']
validation_2features_df['Total revenue'] = validation_df.toPandas()['Total revenue']
testing_2features_df['Total revenue'] = testing_df.toPandas()['Total revenue']


In [26]:
trainging_2features_df_pyspark = spark.createDataFrame(trainging_2features_df)
validation_2features_df_pyspark = spark.createDataFrame(validation_2features_df)
testing_2features_df_pyspark = spark.createDataFrame(testing_2features_df)

In [27]:
from pyspark.ml.classification import LogisticRegression
lr_1 = LogisticRegression().\
    setLabelCol('Total revenue').\
    setFeaturesCol('features')

In [28]:
pipeline_log = Pipeline(stages=[
    feature.VectorAssembler(inputCols=['component1','component2'],
                           outputCol='features')
]).fit(trainging_2features_df_pyspark)

In [29]:
lr_pipeline1 = Pipeline(stages=[pipeline_log, lr_1]).fit(trainging_2features_df_pyspark)

In [30]:
evaluator = evaluation.BinaryClassificationEvaluator(labelCol='Total revenue')
AUC1 = evaluator.evaluate(lr_pipeline1.transform(testing_2features_df_pyspark))

In [31]:
AUC1

0.9647591319515341

In [32]:
1-((testing_2features_df_pyspark.select('Total revenue').toPandas().sum())/len(testing_2features_df_pyspark.select('Total revenue').toPandas()))

Total revenue    0.975731
dtype: float64

In [33]:
prediction = lr_pipeline1.transform(testing_2features_df_pyspark).select('prediction').toPandas()

In [34]:
prediction = prediction.astype('int')

In [35]:
original = testing_2features_df_pyspark.select('Total revenue').toPandas()

In [36]:
combine = pd.concat([prediction,original], axis = 1)

In [37]:
df_confusion = pd.crosstab(original['Total revenue'],prediction['prediction'])

In [38]:
df_confusion

prediction,0,1
Total revenue,,
0,69811,306
1,1499,245


In [41]:
precision = df_confusion[1][1]/(df_confusion[1][1]+df_confusion[1][0])
recall = df_confusion[1][1]/(df_confusion[1][1] + df_confusion[0][1])

In [42]:
precision

0.44464609800362975

In [ ]:
67/(67+105)

In [43]:
F1 = 2 * (precision * recall) / (precision + recall)

In [44]:
F1

0.21350762527233116

In [45]:
G_score = np.sqrt(precision * recall)

In [46]:
G_score

0.24992922623316768